# [1.3.6 Data cleaning & Validation](https://courses.thinkful.com/data-201v1/project/1.3.6)

In [27]:
import pandas as pd
import re
import numpy as np

from scipy.stats.mstats import winsorize

Data cleaning is definitely a "practice makes perfect" skill. Using this dataset of article open-access prices paid by the [WELLCOME Trust between 2012 and 2013](https://www.dropbox.com/s/pl5kcrhs2lyj90m/WELLCOME.zip?dl=0), determine the five most common journals and the total articles for each. Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal . You will need to do considerable data cleaning in order to extract accurate estimates, and may want to look into data [encoding methods](https://stackoverflow.com/questions/2241348/what-is-unicode-utf-8-utf-16) if you get stuck. For a real bonus round, identify the open access prices paid by subject area.

As noted in the previous assignment, don't modify the data directly. Instead, write a cleaning script that will load the raw data and whip it into shape. Jupyter notebooks are a great format for this. Keep a record of your decisions: well-commented code is a must for recording your data cleaning decision-making progress. Submit a link to your script and results below and discuss it with your mentor at your next session.

In [2]:
# import data, 'iso-8859-15'
dt_oac = pd.DataFrame(pd.read_csv('unit_1_data/APCspend2013.csv', encoding='Latin1'))

# Make the columns more aggreeable
dt_oac.rename(columns={'COST (£) charged to Wellcome (inc VAT when charged)':'cost'}, inplace=True)
dt_oac.columns = dt_oac.columns.str.strip().str.lower().str.replace(' ', '_')

# I won't be able to calculate cost when prices are listed as a string included a symbol for pound
dt_oac.cost = dt_oac.cost.str[1:]
dt_oac.cost = dt_oac.cost.replace('$','')

In [3]:
# Look at counts without cleaning
print('unique publishers: {}'.format(len(dt_oac.publisher.unique())))
print('unique journal title: {}'.format(len(dt_oac.journal_title.unique()))) 
len(dt_oac.groupby(['publisher','journal_title']))

unique publishers: 299
unique journal title: 985


1226

In [4]:
# all lower case and replace spaces with underscores
dt_oac['publisher'] = dt_oac['publisher'].str.strip().str.lower()
dt_oac['journal_title'] = dt_oac['journal_title'].str.strip().str.lower()
dt_oac['article_title'] = dt_oac['article_title'].str.strip().str.lower()

# Remove Duplicates
dt_oac = dt_oac.drop_duplicates()
# I wasn't able to use string functions on series with NA and there was one na in journal title
# Dropped that row when i couldn't guess which journal it belonged to
dt_oac = dt_oac[dt_oac.journal_title.notnull()]

In [6]:
# Look at counts without cleaning
print('unique publishers: {}'.format(len(dt_oac.publisher.unique())))
print('unique journal title: {}'.format(len(dt_oac.journal_title.unique()))) 


unique publishers: 253
unique journal title: 894


In [5]:
# Checking for missing values
#df.isnull()
#df.dropna()
#df.fillna(value = 'NA')
dt_oac.isnull().sum()

pmid/pmcid       198
publisher          0
journal_title      0
article_title      0
cost               0
dtype: int64

In [159]:
dt_oac.groupby('publisher').count()

,pmid/pmcid,journal_title,article_title,cost
publisher,,,,
acs,7,7,7,7
acs (amercian chemical society) publications,5,5,5,5
acs publications,2,2,2,2
aga institute,1,1,1,1
ambsb,1,1,1,1
american association of immunologists,2,2,2,2
american chemical society,14,15,15,15
american chemical society publications,1,1,1,1
american college of chest physicians,1,1,1,1


I started by cleaning up some of the most common publishers. Since there are so many similarly titled journals it would help to confirm that whether the journals were atleast from a the same publisher before adjusting how they are written

In [6]:
# the syntax here filters publishers to instances where the string includes any of the substrings listed
# and replaces the entire cell


dt_oac.loc[dt_oac['publisher'].str.contains('acs|american chemical society'),
           'publisher']='american chemical society'
dt_oac.loc[dt_oac['journal_title'].str.contains('j med chem|journal of medical chemistry'),
           'journal_title']='journal of medicinal chemistry'
dt_oac.loc[dt_oac['publisher'].str.contains('wiley|blackwell'), 
           'publisher']='wiley-blackwell'
dt_oac.loc[dt_oac['publisher'].str.contains('wolter|kluwer'), 
           'publisher']='wolters kluwer'
dt_oac.loc[dt_oac['publisher'].str.contains('asbmb|american soc for biochemistry|american society for biochemistry'), 
           'publisher']='american society for biochemistry'
dt_oac.loc[dt_oac['publisher'].str.contains('springer|veriag'), 
           'publisher']='springer-veriag'
dt_oac.loc[dt_oac['publisher'].str.contains('asm'), 
           'publisher']='american society for microbiology'
dt_oac.loc[dt_oac['publisher'].str.contains('the company of bio'), 
           'publisher']='the company of biologists'
dt_oac.loc[dt_oac['publisher'].str.contains('biomed central|bmc'), 
           'publisher']='biomed central limited'
dt_oac.loc[dt_oac['publisher'].str.contains('taylor|francis|t&f'), 
           'publisher']='taylor & francis'
dt_oac.loc[dt_oac['publisher'].str.contains('the end'), 
           'publisher']='the endocrine society'
dt_oac.loc[dt_oac['publisher'].str.contains('american society of heamatology|american society of hamatology|haematology'), 
           'publisher']='american society of hematology'
dt_oac.loc[dt_oac['publisher'].str.contains('american psychiatric'), 
           'publisher']='american psychiatric'
dt_oac.loc[dt_oac['publisher'].str.contains('nature'), 
           'publisher']='nature publishing group'
dt_oac.loc[dt_oac['publisher'].str.contains('elsevier'), 
           'publisher']='elsevier'
dt_oac.loc[dt_oac['publisher'].str.contains('oup|oxford'), 
           'publisher']='oxford university press'
dt_oac.loc[dt_oac['publisher'].str.contains('bmj'), 
           'publisher']='british medical journal'
dt_oac.loc[dt_oac['publisher'].str.contains('plos'), 
           'publisher']='public library of science'

In [7]:
# Look at counts without cleaning
print('unique publishers: {}'.format(len(dt_oac.publisher.unique())))
print('unique journal title: {}'.format(len(dt_oac.journal_title.unique()))) 

unique publishers: 165
unique journal title: 892


To check how the consistant the publisher names are I counted publisher title

In [7]:
dt_oac.groupby(['journal_title']).count()#.sort_values(by='article_title', ascending=False)

,pmid/pmcid,publisher,article_title,cost
journal_title,,,,
academy of nutrition and dietetics,1,1,1,1
acs chemical biology,4,5,5,5
acs chemical neuroscience,1,1,1,1
acs nano,2,2,2,2
"acta crystallographica section d, biological crystallography",1,1,1,1
acta crystallographica section d: biological crystallography,1,1,1,1
acta crystallographica section f: structural biology and crystallization communications,2,2,2,2
"acta crystallographica, section d",1,1,1,1
acta crystallography d,1,1,1,1


In [8]:
dt_oac.loc[dt_oac['journal_title'].str.contains('plosone'), 
           'journal_title']='plos one'
dt_oac.loc[dt_oac['journal_title'].str.contains('trop med int health'), 
           'journal_title']='tropical medicine and international health'
dt_oac.loc[dt_oac['journal_title'].str.contains(
    'acta crystallographica section d|section d|acta d|biological crystallography|acta crystallography d'), 
           'journal_title']='acta crystallographica section d'
dt_oac.loc[dt_oac['journal_title'].str.contains(
    'acta crystallographica section f|section f|acta f|structural biology and crystallization communications'), 
           'journal_title']='acta crystallographica section f'
dt_oac.loc[dt_oac['journal_title'].str.contains('hbm'), 
           'journal_title']='human brain mapping'
dt_oac.loc[dt_oac['journal_title'].str.contains('dev. world bioeth'), 
           'journal_title']='developing world bioethics'
dt_oac.loc[dt_oac['journal_title'].str.contains('bmj'), 
           'journal_title']='british medical journal'
dt_oac.loc[dt_oac['journal_title'].str.contains('acquired immune deficiency|jaids'), 
           'journal_title']='journal of acquired immune deficiency'

In [9]:
dt_oac.loc[dt_oac['journal_title'].str.contains('curr biol.'), 
           'journal_title']='current biology'
dt_oac.loc[dt_oac['journal_title'].str.contains('dev world bioeth'), 
           'journal_title']='developing world bioethics'
dt_oac.loc[dt_oac['journal_title'].str.contains('embo'), 
           'journal_title']='embo molecular medicine'
dt_oac.loc[dt_oac['journal_title'].str.contains('epigen'), 
           'journal_title']='epigenetics'
dt_oac.loc[dt_oac['journal_title'].str.contains('european journal of immunology'), 
           'journal_title']='european journal of immunology'
dt_oac.loc[dt_oac['journal_title'].str.contains('journal of virol'), 
           'journal_title']='journal of virology'
dt_oac.loc[dt_oac['journal_title'].str.contains('mbio'), 
           'journal_title']='matrix biology'
dt_oac.loc[dt_oac['journal_title'].str.contains('molecular bio'), 
           'journal_title']='journal of molecular biology'
dt_oac.loc[dt_oac['journal_title'].str.contains('pnas|proc natl|national academy of sciences'), 
           'journal_title']='proceedings of national academy of sciences'
dt_oac.loc[dt_oac['journal_title'].str.contains('royal society|proceedings b'), 
           'journal_title']='proceedings of royal society'
dt_oac.loc[dt_oac['journal_title'].str.contains('sci rep|scientific reports-11-00861b'), 
           'journal_title']='scientific reports'
dt_oac.loc[dt_oac['journal_title'].str.contains('social psychiatry and psychiatric epidemiol'), 
           'journal_title']='social psychiatry and psychiatric epidemiology'

In [17]:
# Look at counts 
print('unique publishers: {}'.format(len(dt_oac.publisher.unique())))
print('unique journal title: {}'.format(len(dt_oac.journal_title.unique()))) 


unique publishers: 165
unique journal title: 802


In [10]:
#removed 'the' from journal title, entry was inconsistant
dt_oac['journal_title'] = dt_oac['journal_title'].apply(lambda x: x.replace('the ', ''))

#some users had been using 'j ' to represent journal of, replaced to make more consistent
dt_oac['journal_title'] = dt_oac['journal_title'].apply(lambda x: x.replace('j ', 'journal of '))
dt_oac['journal_title'] = dt_oac['journal_title'].apply(lambda x: x.replace('jnl', 'journal of '))

#some users had been using 'j ' to represent journal of, replaced to make more consistent
dt_oac['journal_title'] = dt_oac['journal_title'].apply(lambda x: x.replace('americal', 'american'))
#dt_oac['journal_title'] = dt_oac['journal_title'].apply(lambda x: x.replace('trop ' , 'tropical'))
dt_oac['journal_title'] = dt_oac['journal_title'].apply(lambda x: x.replace('am journal', 'american journal'))
dt_oac['journal_title'] = dt_oac['journal_title'].apply(lambda x: x.replace('vet.', 'veterinary'))
dt_oac['journal_title'] = dt_oac['journal_title'].apply(lambda x: x.replace('biol ', 'biological '))
dt_oac['journal_title'] = dt_oac['journal_title'].apply(lambda x: x.replace('chem ', 'chemistry '))
dt_oac['journal_title'] = dt_oac['journal_title'].apply(lambda x: x.replace('chem. ', 'chemistry '))
dt_oac['journal_title'] =dt_oac['journal_title'].apply(lambda x: x.replace(
    'plos ', 'public library of science '))
dt_oac['journal_title'] = dt_oac['journal_title'].apply(lambda x: x.replace('mol ', 'molecular '))
dt_oac['journal_title'] = dt_oac['journal_title'].apply(lambda x: x.replace('infect ', 'infectious '))
dt_oac['journal_title'] = dt_oac['journal_title'].apply(lambda x: x.replace('eur ', 'european'))
dt_oac['journal_title'] = dt_oac['journal_title'].apply(lambda x: x.replace('&', 'and'))
dt_oac['journal_title'] = dt_oac['journal_title'].apply(lambda x: x.replace('experiements', 'experiments'))
dt_oac['journal_title'] = dt_oac['journal_title'].apply(lambda x: x.replace('ntd', 'neglected tropical disease'))
dt_oac['journal_title'] = dt_oac['journal_title'].apply(lambda x: x.replace('1', 'one'))
# I have to add spaces after these switches for circumstances they are at the end of the journal title
dt_oac['journal_title'] = dt_oac['journal_title'].str.strip().str.lower()


In [19]:
# Look at counts without cleaning
print('unique publishers: {}'.format(len(dt_oac.publisher.unique())))
print('unique journal title: {}'.format(len(dt_oac.journal_title.unique()))) 

unique publishers: 165
unique journal title: 802


### 1. Find top five journals & 2. Find total articles for each

In [12]:
top_journals = pd.DataFrame(dt_oac.groupby(
    ['journal_title'])['article_title'].count().sort_values(ascending=False).head())
top_journals

,article_title
journal_title,
public library of science one,207
journal of biological chemistry,61
proceedings of national academy of sciences,40
neuroimage,29
nucleic acids research,26


### 3. Calculate mean, median and standard deviation of the open-access cost per article for each journal

In [13]:
articles = pd.merge(dt_oac, top_journals, how='inner', on=['journal_title', 'journal_title']).reset_index(drop=True)
articles.cost = articles.cost.astype('float')
pd.pivot_table(articles, columns='journal_title', 
               values='cost', aggfunc=(min, max, np.mean, np.std, np.median)).transpose()

,max,mean,median,min,std
journal_title,,,,,
journal of biological chemistry,999999.00,17784.752623,1324.57,265.67,127856.215554
neuroimage,2503.34,2215.168276,2326.43,1747.16,266.653947
nucleic acids research,2184.00,1149.000000,852.00,710.00,442.940447
proceedings of national academy of sciences,999999.00,25818.861250,742.92,206.32,157981.566997
public library of science one,999999.00,40473.267488,900.14,122.31,193312.883195


**But the max values look like a mistake, will take 2nd highest value for cost for each journal_title**

In [63]:
articles = pd.merge(articles, pd.DataFrame(articles[articles['cost']<5000].groupby('journal_title').max()['cost']), 
         how='inner', on=['journal_title', 'journal_title'])

In [65]:
articles['cost'] = np.where(articles['cost_x']>articles['cost_y'],articles['cost_y'],articles['cost_x'])

In [67]:
pd.pivot_table(articles, columns='journal_title', 
               values='cost', aggfunc=(min, max, np.mean, np.std, np.median)).transpose()

,max,mean,median,min,std
journal_title,,,,,
journal of biological chemistry,2501.07,1432.327541,1324.57,265.67,407.520517
neuroimage,2503.34,2215.168276,2326.43,1747.16,266.653947
nucleic acids research,2184.00,1149.000000,852.00,710.00,442.940447
proceedings of national academy of sciences,2691.68,886.178250,742.92,206.32,536.403873
public library of science one,1785.36,972.935314,900.14,122.31,257.935505
